# Cell 0

# ICE Query Workflow - Investment Intelligence Analysis

**Purpose**: Interactive financial analysis and investment intelligence using LightRAG knowledge graph
**Prerequisites**: Complete knowledge graph built via `ice_building_workflow.ipynb`
**Architecture**: ICE Simplified with 6 LightRAG query modes

## Query Workflow Overview

1. **System Connection** - Connect to built knowledge graph
2. **Portfolio Analysis** - Automated risk and opportunity assessment
3. **Query Mode Testing** - Explore all 6 LightRAG query modes
4. **Investment Intelligence** - Natural language financial queries
5. **Performance Monitoring** - Query metrics and optimization
6. **Results Export** - Save analysis and insights

---

# Cell 1

## 1. System Connection & Readiness Check

In [16]:
# ═══════════════════════════════════════════════════════════════════════════
# Cell 2: Setup and imports
# ═══════════════════════════════════════════════════════════════════════════

import sys
import os
from pathlib import Path
from datetime import datetime
import json

# Add project root to path
project_root = Path.cwd()
sys.path.insert(0, str(project_root))

print(f"🔍 ICE Query Workflow")
print(f"📅 {datetime.now().strftime('%Y-%m-%d %H:%M')}")
print(f"📁 Working Directory: {project_root}")

🔍 ICE Query Workflow
📅 2025-11-08 17:18
📁 Working Directory: /Users/royyeo/Library/CloudStorage/OneDrive-NationalUniversityofSingapore/Capstone Project


In [17]:
# ═══════════════════════════════════════════════════════════════════════════
# Cell 3: Connect to ICE system and verify knowledge graph
# ═══════════════════════════════════════════════════════════════════════════

from updated_architectures.implementation.ice_simplified import create_ice_system

try:
    ice = create_ice_system()
    system_ready = ice.is_ready()
    print(f"✅ ICE System Connected")
    print(f"🧠 LightRAG Status: {'Ready' if system_ready else 'Initializing'}")
    print(f"📊 Architecture: ICE Simplified (2,508 lines)")
    
    if system_ready:
        # Check if knowledge graph has been built
        storage_stats = ice.core.get_storage_stats()
        graph_ready = storage_stats['total_storage_bytes'] > 0
        print(f"🕸️ Knowledge Graph: {'Ready' if graph_ready else 'Not built - run ice_building_workflow.ipynb first'}")
        
        if graph_ready:
            print(f"💾 Graph Size: {storage_stats['total_storage_bytes'] / (1024 * 1024):.2f} MB")
            components_ready = sum(1 for c in storage_stats['components'].values() if c['exists'])
            print(f"📦 Components Ready: {components_ready}/4")
            
            # Phase 2.6.1: Investment signal extraction enabled
            print(f"📧 Investment Signals: EntityExtractor integrated (BUY/SELL ratings, confidence scores)")
        else:
            raise RuntimeError("No knowledge graph found. Please run ice_building_workflow.ipynb first.")
    
except Exception as e:
    print(f"❌ Connection Error: {e}")
    raise  # Let errors surface for proper debugging

INFO:updated_architectures.implementation.ice_simplified:ICE Core initializing with ICESystemManager orchestration
INFO:src.ice_core.ice_system_manager:ICE System Manager initialized with working_dir: ice_lightrag/storage
INFO:updated_architectures.implementation.ice_simplified:✅ ICESystemManager initialized successfully
INFO:imap_email_ingestion_pipeline.entity_extractor:spaCy model loaded successfully
INFO:updated_architectures.implementation.data_ingestion:✅ TickerValidator initialized (false positive filtering)
INFO:src.ice_docling.docling_processor:DoclingProcessor initialized: storage=/Users/royyeo/Library/CloudStorage/OneDrive-NationalUniversityofSingapore/Capstone Project/data/attachments
INFO:updated_architectures.implementation.data_ingestion:✅ DoclingProcessor initialized (97.9% table accuracy)
INFO:updated_architectures.implementation.data_ingestion:✅ BenzingaClient initialized (real-time professional news)
INFO:src.ice_docling.docling_processor:DoclingProcessor initialized

✅ ICE System Connected
🧠 LightRAG Status: Ready
📊 Architecture: ICE Simplified (2,508 lines)
🕸️ Knowledge Graph: Ready
💾 Graph Size: 2.96 MB
📦 Components Ready: 4/4
📧 Investment Signals: EntityExtractor integrated (BUY/SELL ratings, confidence scores)


In [18]:
# ═══════════════════════════════════════════════════════════════════════════
# Cell 4: Portfolio configuration - load from CSV
# ═══════════════════════════════════════════════════════════════════════════

import pandas as pd

try:
    portfolio_df = pd.read_csv('portfolio_holdings.csv')
    # Basic validation
    if portfolio_df.empty:
        raise ValueError("Portfolio CSV is empty")
    if 'ticker' not in portfolio_df.columns:
        raise ValueError("CSV must have 'ticker' column")
    holdings = portfolio_df['ticker'].tolist()
    print(f"📄 Loaded from: portfolio_holdings.csv ({len(holdings)} holdings)")
except FileNotFoundError:
    print("⚠️ portfolio_holdings.csv not found - using defaults")
    holdings = ['NVDA', 'TSMC', 'AMD', 'ASML']

print(f"🎯 Portfolio Configuration")
print(f"━" * 40)
print(f"Holdings: {', '.join(holdings)}")
print(f"Analysis Mode: Investment Intelligence Queries")

# Verify query modes available
if not (ice and ice.core.is_ready()):
    raise RuntimeError("ICE system not available for queries")

available_modes = ice.core.get_query_modes()
print(f"\n🔍 Available Query Modes: {len(available_modes)}")
for mode in available_modes:
    print(f"  ✅ {mode}")

📄 Loaded from: portfolio_holdings.csv (5 holdings)
🎯 Portfolio Configuration
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Holdings: NVDA, TSMC, AMD, ASML, FICO
Analysis Mode: Investment Intelligence Queries

🔍 Available Query Modes: 6
  ✅ naive
  ✅ local
  ✅ global
  ✅ hybrid
  ✅ mix
  ✅ bypass


# Cell 5

### 🔧 Model Provider Configuration

ICE supports **OpenAI** (paid) or **Ollama** (free local) for LLM and embeddings:

#### Option 1: OpenAI (Default - No setup required)
```bash
export OPENAI_API_KEY="sk-..."
```
- **Cost**: ~$5/month for typical usage
- **Quality**: Highest accuracy for entity extraction and reasoning
- **Setup**: Just set API key

#### Option 2: Ollama (Free Local - Requires setup)
```bash
# Set provider
export LLM_PROVIDER="ollama"

# One-time setup:
ollama serve                          # Start Ollama service
ollama pull llama3.1:8b               # Faster model (4.7GB, used in this notebook)
# OR
ollama pull qwen3:30b-32k            # More accurate model (18.5GB, 32k context)
ollama pull nomic-embed-text          # Pull embedding model
```

**Model Selection**:
- **llama3.1:8b** (4.7GB) - Faster, recommended for development and testing (used in Cell 8)
- **qwen3:30b-32k** (18.5GB) - Better accuracy, 32k context for complex financial analysis

**Note**: This notebook uses `llama3.1:8b` for faster iterations. To switch to `qwen3:30b-32k`, update Cell 8.

- **Cost**: $0/month (completely free)
- **Quality**: Good-to-excellent for investment analysis (model-dependent)
- **Setup**: Requires local Ollama installation and model download

#### Option 3: Hybrid (Recommended for cost-conscious users)
```bash
export LLM_PROVIDER="ollama"           # Use Ollama for LLM
export EMBEDDING_PROVIDER="openai"     # Use OpenAI for embeddings
export OPENAI_API_KEY="sk-..."
```
- **Cost**: ~$2/month (embeddings only)
- **Quality**: Balanced - free LLM with high-quality embeddings

**Current configuration will be logged when you connect to ICE system.**

# Cell 6

### 📄 Docling Document Processing (Switchable Architecture)

ICE supports **switchable document processing** for SEC filings and email attachments:

#### Docling Integration (Default - Professional-grade table extraction)
```bash
export USE_DOCLING_SEC=true          # SEC filings: 0% → 97.9% table extraction
export USE_DOCLING_EMAIL=true        # Email attachments: 42% → 97.9% accuracy
```
- **Accuracy**: 97.9% table extraction (vs 42% with PyPDF2)
- **Cost**: $0/month (local execution)
- **Setup**: Models auto-download on first use (~500MB, one-time)

#### Original Implementations (For comparison testing)
```bash
export USE_DOCLING_SEC=false         # Use metadata-only SEC extraction
export USE_DOCLING_EMAIL=false       # Use PyPDF2/openpyxl processors
```
- **Purpose**: A/B testing and backward compatibility
- **Use when**: Comparing extraction accuracy or troubleshooting

**Note**: Docling uses IBM's AI models (DocLayNet, TableFormer, Granite-Docling VLM) for professional-grade document parsing. Both implementations coexist - toggle switches instantly without code changes.

**More info**: See `md_files/DOCLING_INTEGRATION_TESTING.md` for detailed testing procedures.

# Cell 7

### 🌐 Crawl4AI URL Fetching (Reference)

The knowledge graph you're querying was built using ICE's hybrid URL fetching strategy:

- **Default**: Simple HTTP (fast, free) for DBS research URLs, direct PDFs, SEC EDGAR
- **Optional**: Crawl4AI browser automation for complex sites (premium portals, JS-heavy IR sites)

**Configuration** (used during graph building in `ice_building_workflow.ipynb`):
```bash
export USE_CRAWL4AI_LINKS=false      # Default: Simple HTTP only
export USE_CRAWL4AI_LINKS=true       # Enable hybrid routing with Crawl4AI
```

This query workflow uses the **existing graph** - URL fetching only happens during graph building.

**More info**: See `ice_building_workflow.ipynb` Cell 9 for detailed configuration options.

In [19]:
# ═══════════════════════════════════════════════════════════════════════════
# Cell 8: Provider switching - Uncomment ONE option, then restart kernel
# ═══════════════════════════════════════════════════════════════════════════

# ### Option 1: OpenAI ($5/mo, highest quality)
# import os; os.environ['LLM_PROVIDER'] = 'openai' 
# print("✅ Switched to OpenAI")

# ###Option 2: Hybrid ($2/mo, 60% savings, recommended)
# import os; os.environ['LLM_PROVIDER'] = 'ollama'; os.environ['EMBEDDING_PROVIDER'] = 'openai'
# print("✅ Switched to Hybrid")

### Option 3: Full Ollama ($0/mo, faster iterations)
# Using llama3.1:8b (4.7GB) for development speed instead of qwen3:30b-32k (18.5GB)
# To use qwen3 for better accuracy: os.environ['LLM_MODEL'] = 'qwen3:30b-32k'
import os; os.environ['LLM_PROVIDER'] = 'ollama'; os.environ['EMBEDDING_PROVIDER'] = 'ollama'
os.environ['LLM_MODEL'] = 'llama3.1:8b'; print("✅ Switched to Full Ollama (llama3.1:8b - faster)")

✅ Switched to Full Ollama (llama3.1:8b - faster)


In [20]:
# ═══════════════════════════════════════════════════════════════════════════
# Cell 9: Check active LLM provider configuration
# ═══════════════════════════════════════════════════════════════════════════

provider = os.getenv('LLM_PROVIDER', 'openai')
model = os.getenv('LLM_MODEL', 'gpt-4o-mini' if provider == 'openai' else 'qwen3:30b-32k')
embedding = os.getenv('EMBEDDING_PROVIDER', provider)

print("🔧 Active LLM Configuration:")
print(f"  LLM Provider: {provider}")
print(f"  LLM Model: {model}")
print(f"  Embedding Provider: {embedding}")

🔧 Active LLM Configuration:
  LLM Provider: ollama
  LLM Model: llama3.1:8b
  Embedding Provider: ollama


In [21]:
# ═══════════════════════════════════════════════════════════════════════════
# Cell 10: Reinitialize ICE system with updated provider configuration
# ═══════════════════════════════════════════════════════════════════════════

print("🔄 Reinitializing ICE system with new configuration...")
from updated_architectures.implementation.ice_simplified import create_ice_system
ice = create_ice_system()
print("✅ System reinitialized successfully")
print(f"   LightRAG Status: {'Ready' if ice.is_ready() else 'Initializing'}")

INFO:updated_architectures.implementation.ice_simplified:ICE Core initializing with ICESystemManager orchestration
INFO:src.ice_core.ice_system_manager:ICE System Manager initialized with working_dir: ice_lightrag/storage
INFO:updated_architectures.implementation.ice_simplified:✅ ICESystemManager initialized successfully


🔄 Reinitializing ICE system with new configuration...


INFO:imap_email_ingestion_pipeline.entity_extractor:spaCy model loaded successfully
INFO:updated_architectures.implementation.data_ingestion:✅ TickerValidator initialized (false positive filtering)
INFO:src.ice_docling.docling_processor:DoclingProcessor initialized: storage=/Users/royyeo/Library/CloudStorage/OneDrive-NationalUniversityofSingapore/Capstone Project/data/attachments
INFO:updated_architectures.implementation.data_ingestion:✅ DoclingProcessor initialized (97.9% table accuracy)
INFO:updated_architectures.implementation.data_ingestion:✅ BenzingaClient initialized (real-time professional news)
INFO:src.ice_docling.docling_processor:DoclingProcessor initialized: storage=/Users/royyeo/Library/CloudStorage/OneDrive-NationalUniversityofSingapore/Capstone Project/data/attachments
INFO:imap_email_ingestion_pipeline.intelligent_link_processor.IntelligentLinkProcessor:Intelligent Link Processor initialized. Storage path: /Users/royyeo/Library/CloudStorage/OneDrive-NationalUniversityof

✅ System reinitialized successfully
   LightRAG Status: Ready


# Cell 11

## 2. Portfolio Risk & Opportunity Analysis

In [22]:
# ═══════════════════════════════════════════════════════════════════════════
# Cell 12: Automated portfolio analysis using ICE intelligence
# ═══════════════════════════════════════════════════════════════════════════
# NOTE: This operation may take several minutes. If it hangs, restart kernel.

print(f"\n🎯 Automated Portfolio Analysis")
print(f"━" * 40)

if not (ice and ice.core.is_ready()):
    raise RuntimeError("ICE system not ready for portfolio analysis")

try:
    # Execute comprehensive portfolio analysis
    analysis_start = datetime.now()
    analysis = ice.analyze_portfolio(holdings, include_opportunities=True)
    analysis_time = (datetime.now() - analysis_start).total_seconds()
    
    print(f"📊 Analysis completed in {analysis_time:.2f}s")
    print(f"📅 Analysis timestamp: {analysis['timestamp']}")
    
    # Display analysis summary
    summary = analysis.get('summary', {})
    print(f"\n📊 Analysis Summary:")
    print(f"  Total Holdings: {summary.get('total_holdings', len(holdings))}")
    print(f"  Risk Analyses: {summary.get('successful_risk_analyses', 0)}/{len(holdings)}")
    print(f"  Opportunity Analyses: {summary.get('successful_opportunity_analyses', 0)}/{len(holdings)}")
    print(f"  Completion Rate: {summary.get('analysis_completion_rate', 0):.1f}%")
    
except Exception as e:
    print(f"❌ Analysis error: {e}")
    raise  # Re-raise for proper debugging

INFO:updated_architectures.implementation.ice_simplified:Analyzing portfolio risks...
INFO:updated_architectures.implementation.ice_simplified:Analyzing risks for NVDA
INFO:src.ice_lightrag.model_provider:✅ Using OpenAI provider (fallback from Ollama) with temperature=0.0
INFO: [_] Loaded graph from ice_lightrag/storage/graph_chunk_entity_relation.graphml with 80 nodes, 77 edges
INFO:nano-vectordb:Load (80, 1536) data
INFO:nano-vectordb:Init {'embedding_dim': 1536, 'metric': 'cosine', 'storage_file': 'ice_lightrag/storage/vdb_entities.json'} 80 data
INFO:nano-vectordb:Load (77, 1536) data
INFO:nano-vectordb:Init {'embedding_dim': 1536, 'metric': 'cosine', 'storage_file': 'ice_lightrag/storage/vdb_relationships.json'} 77 data
INFO:nano-vectordb:Load (12, 1536) data
INFO:nano-vectordb:Init {'embedding_dim': 1536, 'metric': 'cosine', 'storage_file': 'ice_lightrag/storage/vdb_chunks.json'} 12 data
INFO:src.ice_lightrag.ice_rag_fixed:✅ Pipeline status initialized successfully
INFO:src.ice_l


🎯 Automated Portfolio Analysis
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━


INFO: Query nodes: NVDA, NVIDIA, Supply chain, Regulations, Competition, Financial stability (top_k:40, cosine:0.2)
INFO: Embedding func: 8 new workers initialized (Timeouts: Func: 30s, Worker: 60s, Health Check: 75s)
INFO: Local query: 40 entites, 65 relations
INFO: Query edges: Business risks, Market risks, Supply chain risks, Regulatory risks, Competitive risks, Financial risks (top_k:40, cosine:0.2)
INFO: Global query: 39 entites, 40 relations
INFO: Raw search results: 53 entities, 69 relations, 0 vector chunks
INFO: After truncation: 53 entities, 69 relations
INFO: Selecting 12 from 12 entity-related chunks by vector similarity
INFO: Find no additional relations-related chunks from 69 relations
INFO: Round-robin merged chunks: 12 -> 12 (deduplicated 0)
INFO: Final context: 53 entities, 69 relations, 12 chunks
INFO: Final chunks S+F/O: E9/1 E6/2 E8/3 E8/4 E12/5 E7/6 E3/7 E6/8 E6/9 E7/10 E8/11 E3/12
INFO:  == LLM cache == Query cache hit, using cached response as query result
INFO:s

📊 Analysis completed in 21.69s
📅 Analysis timestamp: 2025-11-08T17:18:50.336425

📊 Analysis Summary:
  Total Holdings: 5
  Risk Analyses: 5/5
  Opportunity Analyses: 5/5
  Completion Rate: 100.0%


In [23]:
# ═══════════════════════════════════════════════════════════════════════════
# Cell 13: Week 4 Feature Demo - Source Attribution
# ═══════════════════════════════════════════════════════════════════════════

print("📚 Week 4: Source Attribution")
print("=" * 60)

result = ice.core.query("Goldman Sachs view on NVDA?", mode='hybrid')

if result.get('status') == 'success':
    print(f"✅ Answer: {result['answer'][:200]}...")
    
    # Display detailed source attribution
    sources = result.get('sources', [])
    if sources:
        print(f"\n📚 Sources Used ({len(sources)} unique):")
        source_groups = {}
        for src in sources:
            src_type = src['type']
            if src_type not in source_groups:
                source_groups[src_type] = []
            source_groups[src_type].append(src['symbol'])
        
        for src_type, symbols in sorted(source_groups.items()):
            symbol_str = ', '.join(sorted(set(symbols)))
            print(f"   • {src_type}: {symbol_str}")
    else:
        print(f"\n⚠️  No explicit SOURCE markers found in answer")
        print(f"   (This may occur if answer synthesizes multiple sources)")
    
    # Display confidence if available
    confidence = result.get('confidence')
    if confidence is not None:
        print(f"\n📊 Confidence: {confidence:.0%}")
    
    print("\n💡 Source attribution enables regulatory compliance and audit trails")
else:
    print(f"❌ Failed: {result.get('message')}")

📚 Week 4: Source Attribution


INFO: Query nodes: NVDA, NVIDIA, Stock recommendation, Market outlook (top_k:40, cosine:0.2)
INFO: Local query: 38 entites, 64 relations
INFO: Query edges: Goldman Sachs, Investment analysis, Stock market (top_k:40, cosine:0.2)
INFO: Global query: 36 entites, 40 relations
INFO: Raw search results: 52 entities, 69 relations, 0 vector chunks
INFO: After truncation: 52 entities, 69 relations
INFO: Selecting 12 from 12 entity-related chunks by vector similarity
INFO: Find no additional relations-related chunks from 69 relations
INFO: Round-robin merged chunks: 12 -> 12 (deduplicated 0)
INFO: Final context: 52 entities, 69 relations, 12 chunks
INFO: Final chunks S+F/O: E9/1 E6/2 E9/3 E9/4 E7/5 E12/6 E3/7 E9/8 E9/9 E3/10 E4/11 E3/12
INFO:  == LLM cache == Query cache hit, using cached response as query result
INFO:src.ice_lightrag.ice_rag_fixed:Parsed context: Retrieved 52 entities, 69 relationships, 12 chunks
INFO:src.ice_lightrag.ice_rag_fixed:Extracted 8 unique sources: ['email', 'entity_

✅ Answer: I do not have enough information to answer....

📚 Sources Used (8 unique):
   • email: CH_HK_ Tencent Music Entertainment (1698 HK)_ Stro
   • entity: DBS, DTD, EN, NOTE, SVIP, TME, TR

📊 Confidence: 70%

💡 Source attribution enables regulatory compliance and audit trails


# Cell 14

## 4c. Week 4 Feature: Source Attribution & Traceability

**Compliance-Ready Intelligence**: Every fact in ICE's responses includes source attribution for regulatory compliance.

**Source Tracking**:
- **Document IDs**: Each fact links to source document
- **Extraction Metadata**: Includes confidence scores and extraction method
- **Citation Chains**: Multi-hop reasoning shows complete inference path

**Business Value**:
- **Regulatory Compliance**: Meet audit and documentation requirements
- **Trust & Transparency**: Users can verify all claims
- **Quality Assurance**: Low-confidence facts are flagged

**Implementation**: Source metadata is automatically captured during graph building and preserved through query processing.

In [24]:
# ═══════════════════════════════════════════════════════════════════════════
# Cell 15: Week 4 Feature Demo - Query Fallback Logic
# ═══════════════════════════════════════════════════════════════════════════

print("🔄 Week 4: Query Fallback Logic")
print("=" * 60)

# Fallback logic is automatic - system gracefully handles mode failures
result = ice.core.query("Portfolio geopolitical risk cascade?", mode='mix')

if result.get('status') == 'success':
    actual_mode = result.get('mode_used', result.get('query_mode', 'mix'))
    print(f"✅ Query successful")
    print(f"   Requested: mix | Actual: {actual_mode}")
    print(f"   Fallback: {'Yes' if actual_mode != 'mix' else 'No'}")
    print(f"   Answer: {result['answer'][:120]}...")
else:
    print(f"❌ Failed: {result.get('message')}")

print("\n💡 Automatic robustness - queries succeed even if advanced modes fail")
print("   (Fallback cascade: mix → hybrid → local)")

🔄 Week 4: Query Fallback Logic


INFO: Query nodes: Risk cascade, Investment strategy, Market volatility, Economic factors (top_k:40, cosine:0.2)
INFO: Local query: 20 entites, 34 relations
INFO: Query edges: Geopolitical risk, Portfolio management, Risk assessment (top_k:40, cosine:0.2)
INFO: Global query: 44 entites, 40 relations
INFO: Naive query: 8 chunks (chunk_top_k:20 cosine:0.2)
INFO: Raw search results: 51 entities, 55 relations, 8 vector chunks
INFO: After truncation: 51 entities, 55 relations
INFO: Selecting 12 from 12 entity-related chunks by vector similarity
INFO: Find no additional relations-related chunks from 55 relations
INFO: Round-robin merged chunks: 20 -> 12 (deduplicated 8)
INFO: Final context: 51 entities, 55 relations, 12 chunks
INFO: Final chunks S+F/O: E7/1 E9/2 E13/3 E7/4 E8/5 E7/6 E8/7 E4/8 E8/9 E3/10 E8/11 E4/12
INFO:  == LLM cache == Query cache hit, using cached response as query result
INFO:src.ice_lightrag.ice_rag_fixed:Parsed context: Retrieved 51 entities, 55 relationships, 12 chunk

✅ Query successful
   Requested: mix | Actual: mix
   Fallback: No
   Answer: I do not have enough information to answer....

💡 Automatic robustness - queries succeed even if advanced modes fail
   (Fallback cascade: mix → hybrid → local)


# Cell 16

## 4b. Week 4 Feature: Automatic Fallback (mix → hybrid → local)

**Robust Query Processing**: ICEQueryProcessor implements automatic fallback logic to ensure queries always succeed.

**Fallback Cascade**:
1. **mix mode** - Attempts combined vector + graph retrieval
2. **hybrid mode** - Falls back to entity-focused analysis
3. **local mode** - Final fallback to simple semantic search

**Benefit**: Users don't need to worry about mode selection - the system automatically finds the best working strategy.

**Implementation**: Fallback logic is internal and automatic. Queries specify desired mode but system handles degradation gracefully.

In [25]:
# ═══════════════════════════════════════════════════════════════════════════
# Cell 17: Week 4 Feature Demo - ICEQueryProcessor Integration
# ═══════════════════════════════════════════════════════════════════════════

print("🚀 Week 4: ICEQueryProcessor Integration")
print("=" * 60)

query = "What are NVDA's supply chain risks through TSMC?"

result_enhanced = ice.core.query(query, mode='hybrid')

if result_enhanced.get('status') == 'success':
    print(f"✅ Enhanced Query Processing")
    print(f"   Answer: {result_enhanced['answer'][:150]}...")
    print(f"   Features: Multi-hop reasoning + confidence scoring")
else:
    print(f"❌ Status: {result_enhanced.get('status')}")
    
print("\n💡 ICEQueryProcessor provides:")
print("   - Multi-hop reasoning")
print("   - Automatic fallback logic")
print("   - Source attribution")

🚀 Week 4: ICEQueryProcessor Integration


INFO: Query nodes: NVIDIA, Taiwan Semiconductor Manufacturing Company, Manufacturing delays, Component shortages, Geopolitical factors (top_k:40, cosine:0.2)
INFO: Local query: 9 entites, 19 relations
INFO: Query edges: Supply chain risks, NVDA, TSMC (top_k:40, cosine:0.2)
INFO: Global query: 43 entites, 37 relations
INFO: Raw search results: 44 entities, 38 relations, 0 vector chunks
INFO: After truncation: 44 entities, 38 relations
INFO: Selecting 12 from 12 entity-related chunks by vector similarity
INFO: Find no additional relations-related chunks from 38 relations
INFO: Round-robin merged chunks: 12 -> 12 (deduplicated 0)
INFO: Final context: 44 entities, 38 relations, 12 chunks
INFO: Final chunks S+F/O: E9/1 E9/2 E7/3 E4/4 E5/5 E7/6 E8/7 E9/8 E5/9 E6/10 E5/11 E3/12
INFO:  == LLM cache == Query cache hit, using cached response as query result
INFO:src.ice_lightrag.ice_rag_fixed:Parsed context: Retrieved 44 entities, 38 relationships, 12 chunks
INFO:src.ice_lightrag.ice_rag_fixed:E

✅ Enhanced Query Processing
   Answer: I do not have enough information to answer....
   Features: Multi-hop reasoning + confidence scoring

💡 ICEQueryProcessor provides:
   - Multi-hop reasoning
   - Automatic fallback logic
   - Source attribution


# Cell 18

## 4a. Week 4 Feature: Enhanced Query Processing (ICEQueryProcessor)

**ICEQueryProcessor Integration**: Week 4 adds sophisticated query processing capabilities to ICE.

**Key Features**:
- **Multi-hop Reasoning**: Follow relationships across 1-3 hops in knowledge graph
- **Automatic Fallback**: Queries gracefully degrade if advanced modes fail (mix → hybrid → local)
- **Source Attribution**: Every fact traces to source documents
- **Confidence Scoring**: Results include reliability metrics

In [26]:
# ═══════════════════════════════════════════════════════════════════════════
# Cell 19: Manual query input - Interactive
# ═══════════════════════════════════════════════════════════════════════════

print("=" * 60)
print(f"📊 Portfolio Holdings: {', '.join(holdings)}")
query = input("💬 Enter your investment question: ")
mode = input("📊 Query mode (naive/local/global/hybrid/mix/bypass) [hybrid]: ") or "hybrid"

print(f"\n🔍 Executing: '{query}' (mode: {mode})")
result = ice.core.query(query, mode=mode)

if result.get('status') == 'success':
    print(f"\n✅ Answer:\n{result['answer']}")
else:
    print(f"\n❌ Error: {result.get('message')}")

📊 Portfolio Holdings: NVDA, TSMC, AMD, ASML, FICO


ERROR:src.ice_lightrag.ice_rag_fixed:LightRAG response structure error: LightRAG response missing required field: data
Traceback (most recent call last):
  File "/Users/royyeo/Library/CloudStorage/OneDrive-NationalUniversityofSingapore/Capstone Project/src/ice_lightrag/ice_rag_fixed.py", line 302, in query
    raise ValueError("LightRAG response missing required field: data")
ValueError: LightRAG response missing required field: data
INFO:src.ice_core.ice_system_manager:ICE query completed: mode=hybrid, graph_context=False
INFO:updated_architectures.implementation.ice_simplified:Query completed: 0 chars, mode: hybrid



🔍 Executing: '' (mode: hybrid)

❌ Error: Invalid response structure: LightRAG response missing required field: data


# Cell 20

## 3. LightRAG Query Mode Testing & Comparison

In [27]:
# ═══════════════════════════════════════════════════════════════════════════
# Cell 21: Test all 6 LightRAG query modes with investment question
# ═══════════════════════════════════════════════════════════════════════════
# NOTE: Testing all modes takes ~2 minutes. Each query may take 15-20s.

# === TENCENT TABLE IMAGE EXTRACTION TEST QUERIES ===
# These queries validate that inline image table extraction is working correctly.
# All answers can ONLY be found in the image table (image001.png) in 'Tencent Q2 2025 Earnings.eml',
# NOT in the email HTML body content.
#
# Test Query 1 (Hard - Multi-row comparison):
# "Which business segment had the highest year-over-year growth rate in Tencent's Q2 2025 earnings?"
# Expected Answer: International Games (35% YoY growth)
# Tests: Multi-row comparison, YoY column extraction, finding maximum value
#
# Test Query 2 (Medium - QoQ analysis):
# "Did Tencent's domestic games revenue increase or decrease from Q1 2025 to Q2 2025?"
# Expected Answer: Decreased by 6% (QoQ: -6%)
# Tests: Single-row QoQ analysis, negative percentage interpretation, sequential comparison
#
# Test Query 3 (Easy - Direct lookup):
# "What was Tencent's operating margin in Q2 2024?"
# Expected Answer: 36.3%
# Tests: Direct lookup, historical period selection, percentage format, margin metric extraction
# =================================================

print(f"\n🔍 Query Mode Testing & Performance Comparison")
print(f"━" * 50)

test_query = "What are the biggest risks for my semiconductor portfolio?"
print(f"📊 Test Query: '{test_query}'")

# Mode descriptions with investment context
modes_with_descriptions = {
    'naive': "Quick factual lookup without graph context relationships",
    'local': "Deep dive into specific entities (companies) and their immediate relationships",
    'global': "Broad market trends and high-level relationship analysis",
    'hybrid': "Complex analysis combining entity details with relationship context",
    'mix': "DEFAULT MODE - Combines vector similarity with graph-based retrieval for balanced results",
    'bypass': "Direct LLM reasoning without knowledge graph retrieval"
}

mode_results = {}

if not (ice and ice.core.is_ready()):
    raise RuntimeError("Cannot test query modes without initialized system")

print(f"\n🧪 Testing All Query Modes:")

for mode, description in modes_with_descriptions.items():
    print(f"\n{mode.upper()} MODE:")
    print(f"  Use Case: {description}")
    
    try:
        query_start = datetime.now()
        result = ice.core.query(test_query, mode=mode)
        query_time = (datetime.now() - query_start).total_seconds()
        
        if result.get('status') == 'success' and result.get('answer'):
            answer = result['answer']
            metrics = result.get('metrics', {})
            
            print(f"  ✅ Response: {answer[:150]}{'...' if len(answer) > 150 else ''}")
            print(f"  ⏱️ Time: {query_time:.2f}s")
            
            if metrics:
                print(f"  📊 Query Mode: {metrics.get('query_mode', mode)}")
                print(f"  📝 Answer Length: {metrics.get('answer_length', len(answer))} chars")
                if 'api_cost_estimated' in metrics:
                    print(f"  💰 Est. Cost: {metrics['api_cost_estimated']}")
            
            # Store for comparison
            mode_results[mode] = {
                'success': True,
                'answer': answer,
                'time': query_time,
                'length': len(answer)
            }
        else:
            print(f"  ❌ Status: {result.get('status', 'unknown')}")
            print(f"  📋 Message: {result.get('message', 'No response available')}")
            mode_results[mode] = {'success': False, 'time': query_time}
            
    except Exception as e:
        print(f"  ❌ Error: {str(e)[:80]}...")
        mode_results[mode] = {'success': False, 'error': str(e)}


🔍 Query Mode Testing & Performance Comparison
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📊 Test Query: 'What are the biggest risks for my semiconductor portfolio?'

🧪 Testing All Query Modes:

NAIVE MODE:
  Use Case: Quick factual lookup without graph context relationships


INFO: Naive query: 12 chunks (chunk_top_k:20 cosine:0.2)
INFO: Final context: 12 chunks
INFO:  == LLM cache == Query cache hit, using cached response as query result
INFO:src.ice_lightrag.ice_rag_fixed:Parsed context: Retrieved 0 entities, 0 relationships, 12 chunks
INFO:src.ice_lightrag.ice_rag_fixed:Extracted 8 unique sources: ['email', 'entity_extraction', 'entity_extraction', 'entity_extraction', 'entity_extraction', 'entity_extraction', 'entity_extraction', 'entity_extraction']
INFO:src.ice_lightrag.ice_rag_fixed:Calculated confidence: 0.70 from 14 scores
INFO:src.ice_core.ice_system_manager:ICE query completed: mode=naive, graph_context=False
INFO:updated_architectures.implementation.ice_simplified:Query completed: 58 chars, mode: naive


  ✅ Response: I do not have enough information to answer.
  ⏱️ Time: 0.76s

LOCAL MODE:
  Use Case: Deep dive into specific entities (companies) and their immediate relationships


INFO: Query nodes: Market volatility, Supply chain issues, Technological advancements, Regulatory changes, Competition (top_k:40, cosine:0.2)
INFO: Local query: 16 entites, 26 relations
INFO: Raw search results: 16 entities, 26 relations, 0 vector chunks
INFO: After truncation: 16 entities, 26 relations
INFO: Selecting 8 from 8 entity-related chunks by vector similarity
INFO: Find no additional relations-related chunks from 26 relations
INFO: Round-robin merged chunks: 8 -> 8 (deduplicated 0)
INFO: Final context: 16 entities, 26 relations, 8 chunks
INFO: Final chunks S+F/O: E3/1 E1/2 E1/3 E2/4 E4/5 E3/6 E1/7 E3/8
INFO:  == LLM cache == Query cache hit, using cached response as query result
INFO:src.ice_lightrag.ice_rag_fixed:Parsed context: Retrieved 16 entities, 26 relationships, 8 chunks
INFO:src.ice_lightrag.ice_rag_fixed:Extracted 8 unique sources: ['email', 'entity_extraction', 'entity_extraction', 'entity_extraction', 'entity_extraction', 'entity_extraction', 'entity_extraction',

  ✅ Response: I do not have enough information to answer.
  ⏱️ Time: 1.48s

GLOBAL MODE:
  Use Case: Broad market trends and high-level relationship analysis


...

  ✅ Response: I do not have enough information to answer.
  ⏱️ Time: 1.85s

MIX MODE:
  Use Case: DEFAULT MODE - Combines vector similarity with graph-based retrieval for balanced results


INFO: Query nodes: Supply chain issues, Technological advancements, Regulatory changes, Market demand, Competition (top_k:40, cosine:0.2)
INFO: Local query: 4 entites, 9 relations
INFO: Query edges: Semiconductor portfolio, Investment risks, Market volatility (top_k:40, cosine:0.2)
INFO: Global query: 44 entites, 40 relations
INFO: Naive query: 12 chunks (chunk_top_k:20 cosine:0.2)
INFO: Raw search results: 45 entities, 45 relations, 12 vector chunks
INFO: After truncation: 45 entities, 45 relations
INFO: Selecting 12 from 12 entity-related chunks by vector similarity
INFO: Find no additional relations-related chunks from 45 relations
INFO: Round-robin merged chunks: 24 -> 12 (deduplicated 12)
INFO: Final context: 45 entities, 45 relations, 12 chunks
INFO: Final chunks S+F/O: E9/1 E6/2 E8/3 E9/4 E3/5 E9/6 E3/7 E6/8 E9/9 E7/10 E6/11 E3/12
INFO:  == LLM cache == Query cache hit, using cached response as query result
INFO:src.ice_lightrag.ice_rag_fixed:Parsed context: Retrieved 45 entitie

  ✅ Response: I do not have enough information to answer.
  ⏱️ Time: 1.72s

BYPASS MODE:
  Use Case: Direct LLM reasoning without knowledge graph retrieval


INFO:src.ice_lightrag.ice_rag_fixed:Parsed context: Retrieved 0 entities, 0 relationships, 0 chunks
INFO:src.ice_core.ice_system_manager:ICE query completed: mode=bypass, graph_context=False
INFO:updated_architectures.implementation.ice_simplified:Query completed: 58 chars, mode: bypass


  ✅ Response: Investing in a semiconductor portfolio can be lucrative, but it also comes with several risks. Here are some of the biggest risks to consider:

1. **M...
  ⏱️ Time: 8.36s


# Cell 22

## 5. Query Evaluation Framework

**Automated Quality Assessment**: Evaluate ICE query responses using defensive metrics.

### Metrics
- **Faithfulness**: How well grounded the answer is in retrieved context
- **Relevancy**: How well the answer addresses the query
- **Entity F1**: Ticker symbol extraction accuracy (if reference provided)

### Approach
- **Rule-based**: No LLM calls, cost-free evaluation
- **Defensive**: Explicit error tracking, no silent failures
- **Small batches**: 3 queries at a time to avoid rate limits

---

In [28]:
# ═══════════════════════════════════════════════════════════════════════════
# Cell 23: Load or create test queries
# ═══════════════════════════════════════════════════════════════════════════

print("📋 Loading Test Queries")
print("=" * 60)

# test_queries_filename = 'test_queries.xlsx' 
test_queries_filename = 'test_queries_1.xlsx' 

try:
    test_queries_df = pd.read_excel(test_queries_filename, engine='openpyxl')
    print(f"✅ Loaded {len(test_queries_df)} queries from {test_queries_filename}")
    
    # Ensure required columns exist
    if 'query_id' not in test_queries_df.columns:
        test_queries_df['query_id'] = [f"Q{i+1}" for i in range(len(test_queries_df))]
    if 'query_text' not in test_queries_df.columns and 'query' in test_queries_df.columns:
        test_queries_df['query_text'] = test_queries_df['query']
    
except FileNotFoundError:
    print("⚠️ test_queries.xlsx not found - creating default test set")
    
    # Default test queries covering key use cases
    default_queries = [
        {
            'query_id': 'Q1',
            'query_text': 'What are the key risks for NVDA?',
            'reference': 'NVDA faces supply chain risks through TSMC, intense competition from AMD and Intel, and regulatory concerns.'
        },
        {
            'query_id': 'Q2',
            'query_text': 'Which semiconductor stocks have positive analyst ratings?',
            'reference': 'NVDA has BUY ratings from Goldman Sachs. TSMC has positive outlook.'
        },
        {
            'query_id': 'Q3',
            'query_text': 'Summarize the latest earnings for my portfolio holdings',
            'reference': 'NVDA reported strong Q2 earnings with revenue growth. TSMC manufacturing capacity expansion.'
        }
    ]
    
    test_queries_df = pd.DataFrame(default_queries)
    print(f"✅ Created {len(test_queries_df)} default test queries")

print(f"\n📊 Test Set Composition:")
print(f"   Queries: {len(test_queries_df)}")
print(f"   With reference: {test_queries_df['reference'].notna().sum() if 'reference' in test_queries_df.columns else 0}")
print(f"\nQueries:")
for _, row in test_queries_df.iterrows():
    print(f"   {row['query_id']}: {row['query_text'][:150]}...")

📋 Loading Test Queries
✅ Loaded 8 queries from test_queries_1.xlsx

📊 Test Set Composition:
   Queries: 8
   With reference: 8

Queries:
   Q1: What is Tencent's operating margin in Q2 2025?...
   Q2: What are Tencent's international games?...
   Q3: What is Tencent Music Entertainment's 2Q 2024 paying users?...
   Q4: What is TME's tiered monetization strategy?...
   Q5: What is TME's music streaming service revenue?...
   Q6: Which business segment had the highest year-over-year growth rate in Tencent's Q2 2025 earnings?...
   Q7: Did Tencent's domestic games revenue increase or decrease from Q1 2025 to Q2 2025?...
   Q8: What was Tencent's operating margin in Q2 2024?...


In [29]:
# ═══════════════════════════════════════════════════════════════════════════
# Cell 24: Initialize evaluator and run evaluation
# ═══════════════════════════════════════════════════════════════════════════

print("\n🔍 Initializing Evaluation Framework")
print("=" * 60)

from src.ice_evaluation import ICEMinimalEvaluator, MinimalEvaluationConfig

# Configure evaluator with defensive settings
config = MinimalEvaluationConfig(
    batch_size=3,  # Small batches to avoid rate limits
    max_retries=2,
    fail_fast=False  # Continue on failures
)

evaluator = ICEMinimalEvaluator(config)
print("✅ Evaluator initialized")
print(f"   Batch size: {config.batch_size}")
print(f"   Fail-fast: {config.fail_fast}")

# Run evaluation
print(f"\n🚀 Running Evaluation on {len(test_queries_df)} queries...")
print(f"   This may take ~{len(test_queries_df) * 10}s ({len(test_queries_df)}q × ~10s/query)")
print("=" * 60)

eval_start = datetime.now()

# Pass ice.core (ICEQueryProcessor) to evaluator
results_df = evaluator.evaluate_queries(test_queries_df, ice.core)

eval_duration = (datetime.now() - eval_start).total_seconds()

print(f"\n✅ Evaluation Complete in {eval_duration:.1f}s")
print(f"   Average: {eval_duration/len(test_queries_df):.1f}s per query")

INFO:src.ice_evaluation.minimal_evaluator:✅ Configuration validated: batch_size=3, model=gpt-4o-mini
INFO:ICEMinimalEvaluator:🔍 Starting evaluation of 8 queries in batches of 3
INFO:ICEMinimalEvaluator:📦 Processing batch 1/3
INFO:ICEMinimalEvaluator:🔎 Evaluating query: Q1



🔍 Initializing Evaluation Framework
✅ Evaluator initialized
   Batch size: 3
   Fail-fast: False

🚀 Running Evaluation on 8 queries...
   This may take ~80s (8q × ~10s/query)


INFO: Query nodes: Financial performance, Quarterly report, Revenue, Profit margin (top_k:40, cosine:0.2)
INFO: Local query: 40 entites, 64 relations
INFO: Query edges: Tencent, Operating margin, Q2 2025 (top_k:40, cosine:0.2)
INFO: Global query: 40 entites, 40 relations
INFO: Raw search results: 56 entities, 68 relations, 0 vector chunks
INFO: After truncation: 56 entities, 68 relations
INFO: Selecting 12 from 12 entity-related chunks by vector similarity
INFO: Find no additional relations-related chunks from 68 relations
INFO: Round-robin merged chunks: 12 -> 12 (deduplicated 0)
INFO: Final context: 56 entities, 68 relations, 12 chunks
INFO: Final chunks S+F/O: E13/1 E7/2 E11/3 E10/4 E9/5 E7/6 E8/7 E5/8 E8/9 E4/10 E5/11 E3/12
INFO:  == LLM cache == Query cache hit, using cached response as query result
INFO:src.ice_lightrag.ice_rag_fixed:Parsed context: Retrieved 56 entities, 68 relationships, 12 chunks
INFO:src.ice_lightrag.ice_rag_fixed:Extracted 8 unique sources: ['email', 'entity


✅ Evaluation Complete in 17.0s
   Average: 2.1s per query


In [30]:
# ═══════════════════════════════════════════════════════════════════════════
# Cell 25: Display Evaluation Results
# ═══════════════════════════════════════════════════════════════════════════
# DEPENDENCY CHECK: This cell requires variables from previous cells
# Run cells in this order: Cell 25 (header) → Cell 24 (load) → Cell 23 (evaluate) → Cell 22 (display)

if 'results_df' not in dir():
    print("⚠️  ERROR: Evaluation results not found")
    print("=" * 60)
    print("\n📋 Section 5 cells must be run in order:")
    print("   1️⃣  Cell 25: Read section header (markdown)")
    print("   2️⃣  Cell 24: Load test queries → creates 'test_queries_df'")
    print("   3️⃣  Cell 23: Run evaluation → creates 'results_df'")
    print("   4️⃣  Cell 22: Display results → uses 'results_df' ⬅️ YOU ARE HERE")
    print("\n⚡ Quick fix: Run Cell 24, then Cell 23, then re-run this cell")
    print("=" * 60)
    raise NameError("Variable 'results_df' not defined. Run evaluation cells in sequence (24 → 23 → 22)")

# ANSI color codes for terminal output
class Colors:
    CYAN = '\033[96m'      # Query text
    GREEN = '\033[92m'     # Successful answers
    YELLOW = '\033[93m'    # "No information" responses
    RESET = '\033[0m'      # Reset to default
    BOLD = '\033[1m'       # Bold text

# Display evaluation results
print("\n📊 Evaluation Results Summary")
print("=" * 60)

# Status breakdown
status_counts = results_df['status'].value_counts()
print(f"\n📈 Status Distribution:")
for status, count in status_counts.items():
    percentage = count / len(results_df) * 100
    icon = "✅" if status == "SUCCESS" else "⚠️" if status == "PARTIAL_SUCCESS" else "❌"
    print(f"   {icon} {status}: {count}/{len(results_df)} ({percentage:.1f}%)")

# Metric scores (only for successful queries)
successful_results = results_df[results_df['status'].isin(['SUCCESS', 'PARTIAL_SUCCESS'])]
if len(successful_results) > 0:
    print(f"\n📊 Metric Scores (n={len(successful_results)}):")

    for metric in ['faithfulness', 'relevancy', 'entity_f1']:
        if metric in successful_results.columns:
            scores = successful_results[metric].dropna()
            if len(scores) > 0:
                print(f"   {metric.title():15s}: μ={scores.mean():.3f}, σ={scores.std():.3f}, min={scores.min():.3f}, max={scores.max():.3f}")

# Show failed queries if any
failed_results = results_df[results_df['status'] == 'FAILURE']
if len(failed_results) > 0:
    print(f"\n⚠️ Failed Queries ({len(failed_results)}):")
    for _, row in failed_results.iterrows():
        print(f"   {row['query_id']}: {row['query_text'][:50]}...")
        if 'failure_reasons' in row and row['failure_reasons']:
            print(f"      Reason: {str(row['failure_reasons'])[:80]}...")

# Display Query-Response pairs table WITH COLOR CODING
print(f"\n📝 Query-Response Pairs:")
print("=" * 60)

# Display query-response pairs vertically for full visibility with colors
qr_cols = ['query_id', 'query_text', 'answer']
if all(col in results_df.columns for col in qr_cols):
    for idx, row in results_df.iterrows():
        # Color query in cyan
        print(f"\n{Colors.CYAN}{Colors.BOLD}{row['query_id']}: {row['query_text']}{Colors.RESET}")
        
        # Color answer based on content
        answer = row['answer']
        if "I do not have enough information" in answer:
            # Yellow for "no information" responses
            print(f"{Colors.YELLOW}{answer}{Colors.RESET}")
        else:
            # Green for substantive answers
            print(f"{Colors.GREEN}{answer}{Colors.RESET}")
        
        if idx < len(results_df) - 1:
            print(f"\n{'-' * 60}")  # Separator between pairs
else:
    print("⚠️  Query-response data not available in results")

# Display detailed results table
print(f"\n📋 Detailed Results:")
print("=" * 60)

# Select columns to display - query_text removed
display_cols = ['query_id', 'status', 'faithfulness', 'relevancy']
if 'entity_f1' in results_df.columns:
    display_cols.append('entity_f1')
display_cols.extend(['execution_time_ms', 'failed_metrics'])

display_df = results_df[display_cols].copy()

# Format numeric columns
for col in ['faithfulness', 'relevancy', 'entity_f1']:
    if col in display_df.columns:
        display_df[col] = display_df[col].apply(lambda x: f"{x:.3f}" if pd.notna(x) else "-")

display_df['execution_time_ms'] = display_df['execution_time_ms'].apply(lambda x: f"{x:.0f}ms")

print(display_df.to_string(index=False))

# Save results
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
results_file = f"evaluation_results_{timestamp}.csv"
results_df.to_csv(results_file, index=False)
print(f"\n💾 Results saved to: {results_file}")



📊 Evaluation Results Summary

📈 Status Distribution:
   ✅ SUCCESS: 8/8 (100.0%)

📊 Metric Scores (n=8):
   Faithfulness   : μ=0.633, σ=0.054, min=0.548, max=0.674
   Relevancy      : μ=0.346, σ=0.315, min=0.000, max=0.786
   Entity_F1      : μ=0.240, σ=0.152, min=0.000, max=0.439

📝 Query-Response Pairs:

Q1: What is Tencent's operating margin in Q2 2025?
In Q2 2025, Tencent Music Entertainment reported an operating margin of 30.7%. This figure is derived from the operating income of RMB 2,593 million against the total revenue of RMB 8,442 million for that quarter. 

### References

- [1] email:CH_HK_ Tencent Music Entertainment (1698 HK)_ Stronger growth with expanding revenue streams  (NOT RATED).eml

------------------------------------------------------------

Q2: What are Tencent's international games?
I do not have enough information to answer that.

------------------------------------------------------------

Q3: What is Tencent Music Entertainment's 2Q 2024 paying users?
In t

# Cell 26

## 📋 Query Workflow Complete

**Summary**: This notebook demonstrates the complete ICE query workflow for investment intelligence.

### Key Capabilities
✅ **Portfolio Analysis**: Automated risk assessment  
✅ **Query Modes**: 6 LightRAG retrieval strategies  
✅ **Natural Language**: Investment intelligence queries  
✅ **Performance**: Real-time analysis with metrics  

### Business Value
- **4,000x Token Efficiency** vs GraphRAG
- **<5 Second Responses** for complex queries
- **99.98% Cost Reduction** vs traditional solutions
- **Institutional Quality** AI investment intelligence

---
**Investment Intelligence Delivered** 🚀

# Cell 27

---

# Cell 28

---